In [1]:
#Import packages.
from pyspark.sql import SparkSession
from pyspark.sql import functions as Func
from pyspark.sql.types import *
from pyspark.sql.window import Window
from pyspark.ml.feature import RegexTokenizer, StopWordsRemover, IDF
from pyspark.ml.linalg import SparseVector, DenseVector, VectorUDT, Vectors
from pyspark.ml.clustering import KMeans, LDA
from nltk.stem import PorterStemmer
import math
import numpy as np
import itertools
#import string

#Create a spark session.
sparkSession = SparkSession.builder.appName("Experiment3").getOrCreate()
#Get default configurations
sparkSession.sparkContext._conf.getAll()
#Update default configurations
conf = sparkSession.sparkContext._conf.setAll([('spark.executor.memory', '16g')\
                                        , ('spark.app.name', 'Spark Updated Conf')\
                                        , ('spark.executor.cores', '8')\
                                        , ('spark.cores.max', '8')\
                                        , ('spark.driver.memory','16g')\
                                        ,('spark.driver.maxResultSize','16g')])
#Stop the current Spark Session
sparkSession.sparkContext.stop()
#Create a Spark Session
sparkSession = SparkSession.builder.config(conf=conf).getOrCreate()

In [2]:
#Loading stopwords_en.txt data into a dataframe.
stopWordsDF = sparkSession.read\
                .load("/home/jovyan/work/stopwords_en.txt", format="text", sep=" ", inferSchema="true", header="false")\
                .toDF('stop_word')
stopWordsDF.show(5, truncate=True)

+---------+
|stop_word|
+---------+
|        a|
|     able|
|    about|
|    above|
|according|
+---------+
only showing top 5 rows



In [3]:
#Loading users_libraries.txt data into a dataframe.
#Defining the column names.
user_columns = ['raw_data']
rawUsersDF = sparkSession.read\
            .load("/home/jovyan/work/mod_users_libraries2.txt", format="text", sep=";",\
                  inferSchema="true", quote='"', header="false")\
            .toDF(*user_columns)

usersDF = rawUsersDF.select(Func.split(rawUsersDF.raw_data, ";").getItem(0).alias("user_hash_id"),\
                           Func.split(rawUsersDF.raw_data, ";").getItem(1).alias("user_library"))
usersDF.show(5)

+--------------------+--------------------+
|        user_hash_id|        user_library|
+--------------------+--------------------+
|28d3f81251d94b097...|3929762,503574,58...|
|d0c9aaa788153daea...|2080631,6343346,5...|
|f05bcffe7951de9e5...|1158654,478707,12...|
|ca4f1ba4094011d9a...|              278019|
|d1d41a15201915503...|6610569,6493797,6...|
+--------------------+--------------------+
only showing top 5 rows



In [4]:
#Loading papers.csv data into a dataframe.
#Defining the column names.
paper_columns = ['paper_id', 'type', 'journal', 'book_title', \
           'series', 'publisher', 'pages', 'volume', \
           'number', 'year', 'month', 'postedate',\
           'address', 'title', 'abstract']

papersDF = sparkSession.read\
            .load("/home/jovyan/work/mod_papers.csv", format="csv", sep=",", inferSchema="true", quote='"', header="false")\
            .toDF(*paper_columns)
papersDF.show(2, truncate=True)

+--------+-------+--------------------+----------+------+---------+-----+------+------+----+-----+-------------------+-------+--------------------+--------------------+
|paper_id|   type|             journal|book_title|series|publisher|pages|volume|number|year|month|          postedate|address|               title|            abstract|
+--------+-------+--------------------+----------+------+---------+-----+------+------+----+-----+-------------------+-------+--------------------+--------------------+
|   80546|article|biology and philo...|      null|  null|     null|   17|    19|     2|2004|  mar|2005-01-26 21:35:21|   null|the arbitrariness...|the genetic code ...|
| 5842862|article|      molecular cell|      null|  null| elsevier|    2|    35|     6|2009|  sep|2009-09-30 17:11:23|   null|how to choose a g...|choosing good pro...|
+--------+-------+--------------------+----------+------+---------+-----+------+------+----+-----+-------------------+-------+--------------------+--------

## Exercise 3.1 (Vector representation for the papers)
To generate the bag-of-words representation for each paper.

In [5]:
#Exercise 3.1
#Concatenate the title and abstract fields of papers together.
textDF = papersDF.select(papersDF.paper_id, Func.concat_ws(" ", papersDF.title, papersDF.abstract).alias("text"))
textDF.show(5)

+--------+--------------------+
|paper_id|                text|
+--------+--------------------+
|   80546|the arbitrariness...|
| 5842862|how to choose a g...|
| 1242600|how to write cons...|
| 3467077|defrosting the di...|
|  309395|why most publishe...|
+--------+--------------------+
only showing top 5 rows



In [6]:
#Exercise 3.1
#Perform tokenization and remove words less than 3 characters.
#Keep the words containing "-" and "_" characters.
reTokenizer = RegexTokenizer(inputCol="text", outputCol="words", minTokenLength=4, pattern="[^-_\\w]")
wordsDF = reTokenizer.transform(textDF)
wordsDF.head()

Row(paper_id=80546, text="the arbitrariness of the genetic code the genetic code has been regarded as arbitrary in the sense that the codon-amino acid assignments could be different than they actually are. this general idea has been spelled out differently by previous, often rather implicit accounts of arbitrariness. they have drawn on the frozen accident theory, on evolutionary contingency, on alternative causal pathways, and on the absence of direct stereochemical interactions between codons and amino acids. it has also been suggested that the arbitrariness of the genetic code justifies attributing semantic information to macromolecules, notably to {dna}. i argue that these accounts of arbitrariness are unsatisfactory. i propose that the code is arbitrary in the sense of jacques monod's concept of chemical arbitrariness: the genetic code is arbitrary in that any codon requires certain chemical and structural properties to specify a particular amino acid, but these properties are not 

In [7]:
#Exercise 3.1
#Function to remove the characters "-" and "_" from words.
def concatConnectedWords(wordList):
    wordSet = set(wordList)
    identity = str.maketrans("", "", "-_")
    wordSet = [word.translate(identity) for word in wordSet]
    return wordSet

udf_concatConnectedWords = Func.udf(concatConnectedWords, ArrayType(StringType()))

In [8]:
#Exercise 3.1
#removing the characters "-" and "_" from words.
removedConnectorsDF = wordsDF.select(wordsDF.paper_id,\
                                     Func.lit(udf_concatConnectedWords(wordsDF.words)).alias("processed_words"))
removedConnectorsDF.take(2)

[Row(paper_id=80546, processed_words=['these', 'actually', 'certain', 'stereochemical', 'notably', 'that', 'concept', 'particular', 'code', 'chemical', 'could', 'they', 'direct', 'propose', 'with', 'about', 'chemistry', 'general', 'regarded', 'notion', 'this', 'nucleic', 'information', 'drawn', 'specify', 'evolution', 'neither', 'absence', 'genetic', 'implicit', 'unsatisfactory', 'previous', 'frozen', 'between', 'alternative', 'amino', 'attributing', 'required', 'codonamino', 'been', 'have', 'structural', 'several', 'properties', 'maintain', 'necessary', 'requires', 'sense', 'hypotheses', 'pathways', 'acids', 'causal', 'codons', 'arbitrary', 'often', 'theory', 'also', 'macromolecules', 'assignments', 'rather', 'accounts', 'contingency', 'acid', 'monod', 'virtue', 'sufficient', 'jacques', 'principle', 'argue', 'idea', 'arbitrariness', 'suggested', 'compatible', 'than', 'recent', 'different', 'semantic', 'codon', 'interactions', 'justifies', 'accident', 'spelled', 'evolutionary', 'differ

In [9]:
#Exercise 3.1
#Creating a list of stop words.
stopWordsList = stopWordsDF.agg(Func.collect_list(stopWordsDF.stop_word)).rdd.flatMap(lambda row: row[0])
stopWordsList.take(10)

['a',
 'able',
 'about',
 'above',
 'according',
 'accordingly',
 'across',
 'actually',
 'after',
 'afterwards']

In [10]:
#Exercise 3.1
#Removing the stop words.
remover = StopWordsRemover(inputCol="processed_words", outputCol="words", stopWords=stopWordsList.collect())
withoutStopWordsDF = remover.transform(removedConnectorsDF)
withoutStopWordsDF.select("words").first()

Row(words=['stereochemical', 'notably', 'concept', 'code', 'chemical', 'direct', 'propose', 'chemistry', 'general', 'regarded', 'notion', 'nucleic', 'information', 'drawn', 'evolution', 'absence', 'genetic', 'implicit', 'unsatisfactory', 'previous', 'frozen', 'alternative', 'amino', 'attributing', 'required', 'codonamino', 'structural', 'properties', 'maintain', 'requires', 'sense', 'hypotheses', 'pathways', 'acids', 'causal', 'codons', 'arbitrary', 'theory', 'macromolecules', 'assignments', 'accounts', 'contingency', 'acid', 'monod', 'virtue', 'sufficient', 'jacques', 'principle', 'argue', 'idea', 'arbitrariness', 'suggested', 'compatible', 'recent', 'semantic', 'codon', 'interactions', 'justifies', 'accident', 'spelled', 'evolutionary', 'differently'])

In [11]:
#Exercise 3.1
#Function to perform stemming.
stemmer = PorterStemmer()

def stemming(wordList):
    wordSet = set(wordList)
    wordSet = [stemmer.stem(word) for word in wordList]
    return sorted(wordSet)

#User defined function to perform stemming.
udf_stemming = Func.udf(stemming, ArrayType(StringType()))

In [12]:
#Exercise 3.1
#Performing stemming.
stemmedWordsDF = withoutStopWordsDF.withColumn("stemmed_words", udf_stemming(withoutStopWordsDF.words))

In [13]:
#Exercise 3.1
#Counting the number of papers (frequency) in which a particular word appears. (Document Frequency)
explodedDF = stemmedWordsDF.select(stemmedWordsDF.paper_id,\
                                   Func.explode(stemmedWordsDF.stemmed_words).alias("single_word"))\
                            .distinct()\
                            .groupBy("single_word")\
                            .agg(Func.count("single_word")\
                                 .alias("df"))
explodedDF.show(10)

+-----------+---+
|single_word| df|
+-----------+---+
|   everyday|  2|
|      input|  2|
|    persist|  1|
| likelihood|  3|
|  geneannot|  1|
|     import|  4|
| photograph|  1|
|       oper|  1|
|    highest|  1|
|      equal|  1|
+-----------+---+
only showing top 10 rows



In [14]:
#Exercise 3.1
#Counting the number of unique papers.
#Setting the upper and lower bounds.
uniqPaperCount = stemmedWordsDF.count()
upperBoundary = 0.1*uniqPaperCount
lowerBoundary = 20
#filter the dataframe to select only words that appear in more than 20 papers and
#less than 10 percent of the total number of papers.
filteredDF = explodedDF.filter((explodedDF.df>=lowerBoundary) & (explodedDF.df<=upperBoundary))
filteredDF.show(10)

+-----------+---+
|single_word| df|
+-----------+---+
|   everyday|  2|
|      input|  2|
| likelihood|  3|
|     import|  4|
|    explain|  5|
|    classif|  3|
|     execut|  2|
|        map|  4|
|  character|  6|
|      uncov|  4|
+-----------+---+
only showing top 10 rows



In [15]:
#Exercise 3.1
#Limiting the number of important terms to 1000.
termsDF = filteredDF.sort("df", ascending=False).limit(1000).select("single_word")
termsDF.show(10)

+-----------+
|single_word|
+-----------+
|  scientist|
|    suggest|
|     articl|
|      major|
|       read|
|     access|
|    process|
|       wide|
|      basic|
|     exampl|
+-----------+
only showing top 10 rows



In [16]:
#Exercise 3.1
#Generating an index to the top 1000 important terms starting with the index 0.
termsDF = termsDF.withColumn("word_id", Func.monotonically_increasing_id())

wordWinSpec = Window.orderBy("word_id")

termsDF = termsDF.withColumn("word_id",Func.row_number().over(wordWinSpec)-1)
termsDF.show(10, truncate=True)

+-----------+-------+
|single_word|word_id|
+-----------+-------+
|    process|      0|
|     access|      1|
|    suggest|      2|
|       wide|      3|
|      major|      4|
|       read|      5|
|     articl|      6|
|  scientist|      7|
|      basic|      8|
|     theori|      9|
+-----------+-------+
only showing top 10 rows



In [17]:
#Exercise 3.1
#Counting the number of times a word appeared in a particular paper (word count per paper) (term frequency).
tempDF = stemmedWordsDF.select(stemmedWordsDF.paper_id,\
                               Func.explode(stemmedWordsDF.stemmed_words).alias("single_word"))\
                        .groupBy("paper_id", "single_word")\
                        .agg(Func.count("single_word").alias("tf"))
tempDF.show(10)

+--------+-----------+---+
|paper_id|single_word| tf|
+--------+-----------+---+
| 1242600|     public|  1|
| 1242600|       talk|  1|
|      99|       call|  1|
|  740681|       imit|  1|
|  740681|      model|  1|
|   99857|      degre|  1|
| 3614773|      studi|  1|
|  117535|   reweight|  1|
| 4131662|   techniqu|  1|
|  115158|     common|  1|
+--------+-----------+---+
only showing top 10 rows



In [18]:
#Exercise 3.1
#Creating a dataframe with term index and count of the word per paper.
#Creating a column containing a mapping of term index -> count per paper.
joinedResult = tempDF.join(termsDF, "single_word").withColumn("map", Func.create_map("word_id", "tf"))\
                    .groupBy("paper_id").agg(Func.collect_list("map").alias("map_list"))
joinedResult.show(10)

+--------+--------------------+
|paper_id|            map_list|
+--------+--------------------+
|  115158|[[0 -> 1], [1 -> ...|
|     101|[[0 -> 1], [3 -> ...|
|  740681|[[0 -> 1], [14 ->...|
| 4778506|[[0 -> 1], [1 -> ...|
|  105906|[[0 -> 1], [69 ->...|
|  212874|[[0 -> 1], [1 -> ...|
|  430834|[[0 -> 2], [13 ->...|
| 5394760|[[0 -> 1], [7 -> ...|
| 4200367|[[0 -> 1], [5 -> ...|
| 3721754|[[0 -> 1], [4 -> ...|
+--------+--------------------+
only showing top 10 rows



In [19]:
#Exercise 3.1
#Function to create a sparse vector for each paper.
def toSparse(mapList):
    index = []
    value = []
    for map_val in mapList:
        for idx, val in map_val.items():
            index.append(idx)
            value.append(val)
    return SparseVector(1000, index, value)

#Creating a user defined function.
udf_toSparse = Func.udf(toSparse, VectorUDT())

In [20]:
#Exercise 3.1
#Creating the bag of words dataframe.
featurizedDataDF = joinedResult.select("paper_id",\
                                   udf_toSparse(joinedResult.map_list)\
                                   .alias("tf_vector"))
featurizedDataDF.first()

Row(paper_id=115158, tf_vector=SparseVector(1000, {0: 1.0, 1: 1.0, 12: 1.0, 13: 1.0, 21: 1.0, 22: 2.0, 24: 1.0, 27: 1.0, 31: 1.0, 37: 1.0, 43: 1.0, 51: 1.0, 55: 1.0, 62: 1.0, 69: 1.0, 70: 1.0, 76: 1.0, 79: 1.0, 80: 1.0, 84: 1.0, 90: 1.0, 92: 1.0, 95: 1.0, 105: 2.0, 109: 1.0, 110: 1.0, 111: 2.0, 113: 1.0, 117: 1.0, 120: 1.0, 127: 1.0, 130: 1.0, 145: 1.0, 162: 1.0, 164: 1.0, 169: 1.0, 174: 1.0, 180: 1.0, 202: 1.0, 209: 1.0, 219: 1.0, 222: 1.0, 224: 1.0, 239: 1.0, 245: 1.0, 246: 1.0, 258: 1.0, 263: 2.0, 270: 1.0, 275: 1.0, 317: 2.0, 329: 1.0, 336: 1.0, 339: 1.0, 355: 1.0, 364: 1.0, 369: 1.0, 377: 1.0, 384: 1.0, 387: 1.0, 405: 1.0, 411: 1.0, 418: 2.0, 423: 1.0, 436: 1.0, 440: 1.0, 450: 1.0, 454: 1.0, 459: 1.0, 471: 1.0, 500: 1.0, 534: 1.0, 570: 1.0, 583: 1.0, 584: 1.0, 596: 1.0, 614: 1.0}))

## Exercise 3.2 (TF-IDF representation for the papers)


Calculating TF and IDF using functionality provided by pyspark.ml.feature.IDF

In [21]:
#Exercise 3.2
#Initializing the IDF object.
sparkMlIdf = IDF(inputCol="tf_vector", outputCol="idf_vector")
#Training the data and creating a model.
sparkMlIdfModel = sparkMlIdf.fit(featurizedDataDF)
#Adding the output produced from IDF to the dataset as a separate column.
IdfData = sparkMlIdfModel.transform(featurizedDataDF)
IdfData.first()

Row(paper_id=115158, tf_vector=SparseVector(1000, {0: 1.0, 1: 1.0, 12: 1.0, 13: 1.0, 21: 1.0, 22: 2.0, 24: 1.0, 27: 1.0, 31: 1.0, 37: 1.0, 43: 1.0, 51: 1.0, 55: 1.0, 62: 1.0, 69: 1.0, 70: 1.0, 76: 1.0, 79: 1.0, 80: 1.0, 84: 1.0, 90: 1.0, 92: 1.0, 95: 1.0, 105: 2.0, 109: 1.0, 110: 1.0, 111: 2.0, 113: 1.0, 117: 1.0, 120: 1.0, 127: 1.0, 130: 1.0, 145: 1.0, 162: 1.0, 164: 1.0, 169: 1.0, 174: 1.0, 180: 1.0, 202: 1.0, 209: 1.0, 219: 1.0, 222: 1.0, 224: 1.0, 239: 1.0, 245: 1.0, 246: 1.0, 258: 1.0, 263: 2.0, 270: 1.0, 275: 1.0, 317: 2.0, 329: 1.0, 336: 1.0, 339: 1.0, 355: 1.0, 364: 1.0, 369: 1.0, 377: 1.0, 384: 1.0, 387: 1.0, 405: 1.0, 411: 1.0, 418: 2.0, 423: 1.0, 436: 1.0, 440: 1.0, 450: 1.0, 454: 1.0, 459: 1.0, 471: 1.0, 500: 1.0, 534: 1.0, 570: 1.0, 583: 1.0, 584: 1.0, 596: 1.0, 614: 1.0}), idf_vector=SparseVector(1000, {0: 2.1972, 1: 2.1972, 12: 2.2925, 13: 2.2925, 21: 2.2925, 22: 4.7958, 24: 2.3979, 27: 2.3979, 31: 2.3979, 37: 2.3979, 43: 2.5157, 51: 2.5157, 55: 2.5157, 62: 2.5157, 69: 2

In [22]:
#Exercise 3.2
#Function to multiply 2 sparse vectors.
def multiplySparseVec(x_vec, y_vec):
    result = np.multiply(x_vec, y_vec).tolist()
    vector_args = len(result), [i for i, x in enumerate(result) if x != 0], [x for x in result if x != 0] 
    return Vectors.sparse(*vector_args)

#Creating a user defined function.
udf_multiplySparseVec = Func.udf(multiplySparseVec, VectorUDT())

In [23]:
#Exercise 3.2
#Calculating the value of TF-IDF.
paperTfIdf = IdfData.withColumn("tf_idf", udf_multiplySparseVec(IdfData.tf_vector, IdfData.idf_vector))
paperTfIdf.first()

Row(paper_id=115158, tf_vector=SparseVector(1000, {0: 1.0, 1: 1.0, 12: 1.0, 13: 1.0, 21: 1.0, 22: 2.0, 24: 1.0, 27: 1.0, 31: 1.0, 37: 1.0, 43: 1.0, 51: 1.0, 55: 1.0, 62: 1.0, 69: 1.0, 70: 1.0, 76: 1.0, 79: 1.0, 80: 1.0, 84: 1.0, 90: 1.0, 92: 1.0, 95: 1.0, 105: 2.0, 109: 1.0, 110: 1.0, 111: 2.0, 113: 1.0, 117: 1.0, 120: 1.0, 127: 1.0, 130: 1.0, 145: 1.0, 162: 1.0, 164: 1.0, 169: 1.0, 174: 1.0, 180: 1.0, 202: 1.0, 209: 1.0, 219: 1.0, 222: 1.0, 224: 1.0, 239: 1.0, 245: 1.0, 246: 1.0, 258: 1.0, 263: 2.0, 270: 1.0, 275: 1.0, 317: 2.0, 329: 1.0, 336: 1.0, 339: 1.0, 355: 1.0, 364: 1.0, 369: 1.0, 377: 1.0, 384: 1.0, 387: 1.0, 405: 1.0, 411: 1.0, 418: 2.0, 423: 1.0, 436: 1.0, 440: 1.0, 450: 1.0, 454: 1.0, 459: 1.0, 471: 1.0, 500: 1.0, 534: 1.0, 570: 1.0, 583: 1.0, 584: 1.0, 596: 1.0, 614: 1.0}), idf_vector=SparseVector(1000, {0: 2.1972, 1: 2.1972, 12: 2.2925, 13: 2.2925, 21: 2.2925, 22: 4.7958, 24: 2.3979, 27: 2.3979, 31: 2.3979, 37: 2.3979, 43: 2.5157, 51: 2.5157, 55: 2.5157, 62: 2.5157, 69: 2

Calculating TF and IDF WITHOUT using functionality provided by pyspark.ml.feature.IDF

In [24]:
#Exercise 3.2
#Creating a dataframe with the terms, ID and document-frequency.
termsWithDF = termsDF.join(filteredDF, "single_word")
termsWithDF.show(10)

+-----------+-------+---+
|single_word|word_id| df|
+-----------+-------+---+
|    process|      0| 10|
|     access|      1| 10|
|    suggest|      2| 10|
|       wide|      3| 10|
|      major|      4| 10|
|       read|      5| 10|
|     articl|      6| 10|
|  scientist|      7| 10|
|      basic|      8| 10|
|     theori|      9|  9|
+-----------+-------+---+
only showing top 10 rows



In [25]:
#Exercise 3.2
#creating a document frequency array where index of array element is equal to word_id
docFreqList = termsWithDF.select("word_id", "df").orderBy("word_id", ascending=True).rdd.map(lambda x: x[1]).collect()
docFreqList[0:5]

[10, 10, 10, 10, 10]

In [26]:
#Exercise 3.2
#Function to calculate the IDF.
def calcIdf(m, docFreq):
    Idf = math.log((m+1)/(docFreq+1))
    return Idf

#Function to create the IDF vector.
def getIdfSparseVec(m, tf_vector, doc_freq_ls):
    index = []
    value = []
    #retrieving active indices
    active_idx = tf_vector.indices

    for idx in active_idx:
        doc_freq = doc_freq_ls[idx]
        index.append(idx)
        value.append(calcIdf(m, doc_freq))
    return SparseVector(1000, index, value)

udf_getIdfSparseVec = Func.udf(lambda x, y: getIdfSparseVec(x, y, docFreqList), VectorUDT())

In [27]:
#Exercise 3.2
#Creating a new dataframe with IDF values.
manualIdf = featurizedDataDF.withColumn("idf_vector", udf_getIdfSparseVec(Func.lit(uniqPaperCount),\
                                                                           featurizedDataDF.tf_vector))
manualIdf.first()

Row(paper_id=115158, tf_vector=SparseVector(1000, {0: 1.0, 1: 1.0, 12: 1.0, 13: 1.0, 21: 1.0, 22: 2.0, 24: 1.0, 27: 1.0, 31: 1.0, 37: 1.0, 43: 1.0, 51: 1.0, 55: 1.0, 62: 1.0, 69: 1.0, 70: 1.0, 76: 1.0, 79: 1.0, 80: 1.0, 84: 1.0, 90: 1.0, 92: 1.0, 95: 1.0, 105: 2.0, 109: 1.0, 110: 1.0, 111: 2.0, 113: 1.0, 117: 1.0, 120: 1.0, 127: 1.0, 130: 1.0, 145: 1.0, 162: 1.0, 164: 1.0, 169: 1.0, 174: 1.0, 180: 1.0, 202: 1.0, 209: 1.0, 219: 1.0, 222: 1.0, 224: 1.0, 239: 1.0, 245: 1.0, 246: 1.0, 258: 1.0, 263: 2.0, 270: 1.0, 275: 1.0, 317: 2.0, 329: 1.0, 336: 1.0, 339: 1.0, 355: 1.0, 364: 1.0, 369: 1.0, 377: 1.0, 384: 1.0, 387: 1.0, 405: 1.0, 411: 1.0, 418: 2.0, 423: 1.0, 436: 1.0, 440: 1.0, 450: 1.0, 454: 1.0, 459: 1.0, 471: 1.0, 500: 1.0, 534: 1.0, 570: 1.0, 583: 1.0, 584: 1.0, 596: 1.0, 614: 1.0}), idf_vector=SparseVector(1000, {0: 2.2172, 1: 2.2172, 12: 2.3125, 13: 2.3125, 21: 2.3125, 22: 2.4179, 24: 2.4179, 27: 2.4179, 31: 2.4179, 37: 2.4179, 43: 2.5357, 51: 2.5357, 55: 2.5357, 62: 2.5357, 69: 2

In [28]:
#Exercise 3.2
#Calculating the value of TF-IDF.
manualTfIdf = manualIdf.withColumn("tf_idf", udf_multiplySparseVec(manualIdf.tf_vector, manualIdf.idf_vector))
manualTfIdf.first()

Row(paper_id=115158, tf_vector=SparseVector(1000, {0: 1.0, 1: 1.0, 12: 1.0, 13: 1.0, 21: 1.0, 22: 2.0, 24: 1.0, 27: 1.0, 31: 1.0, 37: 1.0, 43: 1.0, 51: 1.0, 55: 1.0, 62: 1.0, 69: 1.0, 70: 1.0, 76: 1.0, 79: 1.0, 80: 1.0, 84: 1.0, 90: 1.0, 92: 1.0, 95: 1.0, 105: 2.0, 109: 1.0, 110: 1.0, 111: 2.0, 113: 1.0, 117: 1.0, 120: 1.0, 127: 1.0, 130: 1.0, 145: 1.0, 162: 1.0, 164: 1.0, 169: 1.0, 174: 1.0, 180: 1.0, 202: 1.0, 209: 1.0, 219: 1.0, 222: 1.0, 224: 1.0, 239: 1.0, 245: 1.0, 246: 1.0, 258: 1.0, 263: 2.0, 270: 1.0, 275: 1.0, 317: 2.0, 329: 1.0, 336: 1.0, 339: 1.0, 355: 1.0, 364: 1.0, 369: 1.0, 377: 1.0, 384: 1.0, 387: 1.0, 405: 1.0, 411: 1.0, 418: 2.0, 423: 1.0, 436: 1.0, 440: 1.0, 450: 1.0, 454: 1.0, 459: 1.0, 471: 1.0, 500: 1.0, 534: 1.0, 570: 1.0, 583: 1.0, 584: 1.0, 596: 1.0, 614: 1.0}), idf_vector=SparseVector(1000, {0: 2.2172, 1: 2.2172, 12: 2.3125, 13: 2.3125, 21: 2.3125, 22: 2.4179, 24: 2.4179, 27: 2.4179, 31: 2.4179, 37: 2.4179, 43: 2.5357, 51: 2.5357, 55: 2.5357, 62: 2.5357, 69: 2

## Exercise 3.3 (Clustering)

a) Calculating user profile for each user as the summation of the TF-IDF vectors of the papers.

In [29]:
#Exercise 3.3 a)
#Splitting the list of paper_ids into individual rows.
explodUsersDF = usersDF.select(usersDF.user_hash_id,\
                               Func.explode(Func.split(usersDF.user_library, ","))\
                               .alias("paper_id"))
explodUsersDF.show(5, truncate=True)

+--------------------+--------+
|        user_hash_id|paper_id|
+--------------------+--------+
|28d3f81251d94b097...| 3929762|
|28d3f81251d94b097...|  503574|
|28d3f81251d94b097...| 5819422|
|28d3f81251d94b097...| 4238883|
|28d3f81251d94b097...| 5788061|
+--------------------+--------+
only showing top 5 rows



In [30]:
#Exercise 3.3 a)
#Creating a dataframe with user_hash_id as well as paper_id, tf, idf, tf-idf information.
joinUserPaperTfIdf = explodUsersDF.join(paperTfIdf, "paper_id")
joinUserPaperTfIdf.show(5)

+--------+--------------------+--------------------+--------------------+--------------------+
|paper_id|        user_hash_id|           tf_vector|          idf_vector|              tf_idf|
+--------+--------------------+--------------------+--------------------+--------------------+
|  740681|b656009a6efdc8b1a...|(1000,[0,14,22,37...|(1000,[0,14,22,37...|(1000,[0,14,22,37...|
| 4778506|a06e5e3519a8d90a8...|(1000,[0,1,5,33,3...|(1000,[0,1,5,33,3...|(1000,[0,1,5,33,3...|
|  105906|d2e2a082f925839b3...|(1000,[0,69,292,5...|(1000,[0,69,292,5...|(1000,[0,69,292,5...|
|  105906|f1e1cd4ff25018273...|(1000,[0,69,292,5...|(1000,[0,69,292,5...|(1000,[0,69,292,5...|
|  212874|a06e5e3519a8d90a8...|(1000,[0,1,14,37,...|(1000,[0,1,14,37,...|(1000,[0,1,14,37,...|
+--------+--------------------+--------------------+--------------------+--------------------+
only showing top 5 rows



In [31]:
#Exercise 3.3 a)
#Grouping the tf-idf vectors with respect to the user.
userTfIdfList = joinUserPaperTfIdf.groupBy("user_hash_id").agg(Func.collect_list("tf_idf").alias("tf_idf_list"))
userTfIdfList.show(10)

+--------------------+--------------------+
|        user_hash_id|         tf_idf_list|
+--------------------+--------------------+
|f1e1cd4ff25018273...|[(1000,[0,69,292,...|
|cf9c7f356092c34be...|[(1000,[3,19,28,3...|
|f3da73f972caebbe0...|[(1000,[7,70,77,2...|
|d46f0fdd50ec8aa79...|[(1000,[2,9,21,32...|
|d503571e44a0373eb...|[(1000,[2,9,21,32...|
|ee1dfee93ebeadade...|[(1000,[35,318,37...|
|f05bcffe7951de9e5...|[(1000,[1,30,47,7...|
|dad19fd5a1bead8a2...|[(1000,[1,7,15,67...|
|d2e2a082f925839b3...|[(1000,[0,69,292,...|
|cbd4a69e4b3ed3472...|[(1000,[0,1,14,37...|
+--------------------+--------------------+
only showing top 10 rows



In [32]:
#Exercise 3.3 a)
#Function to add elements of a list of sparse vectors.
def addSparseVec(vector_list):
    result = SparseVector(1000, list(range(0, 1000)), np.zeros(1000))
    for vector in vector_list:
        result = np.add(result, vector).tolist()

    vector_args = len(result), [i for i, x in enumerate(result) if x != 0], [x for x in result if x != 0] 
    return Vectors.sparse(*vector_args)
    
udf_addSparseVec = Func.udf(addSparseVec, VectorUDT())

In [33]:
#Exercise 3.3 a)
#Creating the user profile.
userProfileDF = userTfIdfList.withColumn("features", udf_addSparseVec(userTfIdfList.tf_idf_list))
userFeaturesDF = userProfileDF.select("user_hash_id", "features")
userFeaturesDF.first()

Row(user_hash_id='f1e1cd4ff25018273aafc0c68fbb5a2f', features=SparseVector(1000, {0: 2.1972, 7: 2.1972, 21: 2.2925, 25: 2.3979, 38: 2.3979, 41: 2.5157, 43: 2.5157, 59: 2.5157, 69: 2.6492, 84: 2.6492, 96: 2.8034, 102: 2.8034, 109: 2.8034, 110: 2.8034, 139: 2.8034, 143: 2.8034, 157: 2.9857, 163: 2.9857, 173: 2.9857, 177: 2.9857, 179: 2.9857, 199: 2.9857, 247: 3.2088, 274: 3.2088, 287: 3.2088, 292: 3.2088, 297: 3.2088, 323: 3.2088, 450: 3.4965, 504: 3.4965, 516: 13.986, 548: 3.4965, 570: 3.4965, 597: 3.4965, 601: 3.4965, 603: 3.4965, 609: 3.4965}))

b) Applying the K-Means algorithm to cluster the users in 50 clusters given their profiles.

In [52]:
#Exercise 3.3 b)
#Initializing a Kmeans object.
kmeans50 = KMeans(featuresCol="features", k=50, maxIter=10)
#Creating the model.
userClusterModel = kmeans50.fit(userFeaturesDF.select("features"))

c) Calculating the Davies-Bouldin index for the 50 Generated clusters.

In [35]:
#Exercise 3.3 c)
#Getting the list of centroids.
centroids50_list = userClusterModel.clusterCenters()
centroids50_list[0:1]

[array([ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  2.29253476,
         0.        ,  0.        ,  2.29253476,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         2.39789527,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        , 10.06271323,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  2.6

In [36]:
#Exercise 3.3 c)
#Creating a dataframe with the predictions of the KMeans algo.
userTransformed = userClusterModel.transform(userFeaturesDF)
userTransformed.show(5)

+--------------------+--------------------+----------+
|        user_hash_id|            features|prediction|
+--------------------+--------------------+----------+
|f1e1cd4ff25018273...|(1000,[0,7,21,25,...|         1|
|cf9c7f356092c34be...|(1000,[3,19,28,31...|         2|
|f3da73f972caebbe0...|(1000,[7,70,77,25...|         3|
|d46f0fdd50ec8aa79...|(1000,[2,9,21,32,...|         4|
|d503571e44a0373eb...|(1000,[2,9,21,32,...|         4|
+--------------------+--------------------+----------+
only showing top 5 rows



In [37]:
#Exercise 3.3 c)
#Function to calculate the euclidean distance between 2 points.
def calcDistance(point_a, point_b):
    return(np.sqrt(np.sum(np.square(np.subtract(point_a, point_b)))))

#Function to calculate the inter-cluster distances between a set of points and its centroid.
def interClusterDist(prediction, user_points, centroids_list):
    distances_list = []
    index = int(prediction)
    centroid = centroids_list[index]
    for point in user_points:
        distances_list.append(calcDistance(point, centroid))
    average_distance = np.average(distances_list)
    return(float(average_distance))    

In [38]:
#Creating a user-defined function
udf_calcInterClustDist50 = Func.udf(lambda x, y: interClusterDist(x, y, centroids50_list), FloatType())

In [39]:
#Exercise 3.3 c)
#Grouping the user features with respect to the prediction.
groupedUserProfDF = userTransformed.groupBy("prediction").agg(Func.collect_list("features").alias("user_points"))
groupedUserProfDF.show(5)              

+----------+--------------------+
|prediction|         user_points|
+----------+--------------------+
|        26|[(1000,[0,35,69,2...|
|        12|[(1000,[0,4,10,11...|
|        22|[(1000,[0,4,6,7,1...|
|         1|[(1000,[0,7,21,25...|
|        13|[(1000,[7,39,67,7...|
+----------+--------------------+
only showing top 5 rows



In [40]:
#Exercise 3.3 c)
#Creating a dataframe containing the inter-cluster distance values.
interClustDist50DF = groupedUserProfDF.withColumn("inter_cluster_dist",\
                                                udf_calcInterClustDist50(groupedUserProfDF.prediction,\
                                                                         groupedUserProfDF.user_points))
interClustDist50DF.show(5)

+----------+--------------------+------------------+
|prediction|         user_points|inter_cluster_dist|
+----------+--------------------+------------------+
|        26|[(1000,[0,35,69,2...|               0.0|
|        12|[(1000,[0,4,10,11...|               0.0|
|        22|[(1000,[0,4,6,7,1...|               0.0|
|         1|[(1000,[0,7,21,25...|               0.0|
|        13|[(1000,[7,39,67,7...|               0.0|
+----------+--------------------+------------------+
only showing top 5 rows



In [41]:
#Exercise 3.3 c)
#Creating a list of inter-cluster distances.
interClust50List = interClustDist50DF.select("prediction", "inter_cluster_dist")\
                                    .orderBy("prediction", ascending=True)\
                                    .rdd.map(lambda x: x[1]).collect()
#interClust50List

In [42]:
#Exercise 3.3 c)
#Function to calculate the Davies-Boulding index.
def calcDBIndex(centroids_list, inter_clust_list):
    distance = []
    counter = 0
    #calculate Davies-Bouldin value for pairs of clusters.
    while counter < len(centroids_list):
        other_counter = counter + 1
        other_centroids = list(itertools.islice(centroids_list, counter+1, None))
        for other_centroid in other_centroids:
            inter_clust_pair = inter_clust_list[counter]+inter_clust_list[other_counter]
            #Calculate intra cluster distance (distance between 2 clusters)
            intra_clust_pair = calcDistance(centroids_list[counter], other_centroid)
            distance.append(inter_clust_pair/intra_clust_pair)
            other_counter += 1    
        counter += 1
        
    #calculate the Davies-Bouldin index.
    DB_index = sum(distance)/len(centroids_list)
    
    return(DB_index)    

In [43]:
#Exercise 3.3 c)
#Davies-Bouldin Index for a cluster of size 50
calcDBIndex(centroids50_list, interClust50List)

1.2818357500840474e-16

d) Calculating the Davies-Bouldin for the 10 generated clusters.

In [44]:
#Exercise 3.3 d)
#Initializing a Kmeans object.
kmeans10 = KMeans(featuresCol="features", k=10, maxIter=10)
#Creating a model.
userCluster10Model = kmeans10.fit(userFeaturesDF.select("features"))

In [45]:
#Exercise 3.3 d)
#Getting the list of centroids.
centroids10_list = userCluster10Model.clusterCenters()

In [46]:
#Exercise 3.3 d)
#Creating a dataframe with the predictions of the KMeans algo.
user10Transformed = userCluster10Model.transform(userFeaturesDF)
user10Transformed.show(5)

+--------------------+--------------------+----------+
|        user_hash_id|            features|prediction|
+--------------------+--------------------+----------+
|f1e1cd4ff25018273...|(1000,[0,7,21,25,...|         7|
|cf9c7f356092c34be...|(1000,[3,19,28,31...|         7|
|f3da73f972caebbe0...|(1000,[7,70,77,25...|         7|
|d46f0fdd50ec8aa79...|(1000,[2,9,21,32,...|         7|
|d503571e44a0373eb...|(1000,[2,9,21,32,...|         7|
+--------------------+--------------------+----------+
only showing top 5 rows



In [47]:
#Exercise 3.3 d)
#Grouping the user features with respect to the prediction.
groupedUserProf10DF = user10Transformed.groupBy("prediction").agg(Func.collect_list("features").alias("user_points"))
groupedUserProf10DF.show(5)

+----------+--------------------+
|prediction|         user_points|
+----------+--------------------+
|         1|[(1000,[1,3,6,8,1...|
|         6|[(1000,[1,3,7,11,...|
|         3|[(1000,[0,1,2,3,4...|
|         5|[(1000,[0,13,21,2...|
|         9|[(1000,[5,6,46,47...|
+----------+--------------------+
only showing top 5 rows



In [48]:
#Exercise 3.3 d)
#Creating a user defined function
udf_calcInterClustDist10 = Func.udf(lambda x, y: interClusterDist(x, y, centroids10_list), FloatType())

In [49]:
#Exercise 3.3 d)
#Creating a dataframe containing the inter-cluster distance values.
interClustDist10DF = groupedUserProf10DF.withColumn("inter_cluster_dist",\
                                                udf_calcInterClustDist10(groupedUserProf10DF.prediction,\
                                                                        groupedUserProf10DF.user_points))
interClustDist10DF.show(5)

+----------+--------------------+------------------+
|prediction|         user_points|inter_cluster_dist|
+----------+--------------------+------------------+
|         1|[(1000,[1,3,6,8,1...|               0.0|
|         6|[(1000,[1,3,7,11,...|               0.0|
|         3|[(1000,[0,1,2,3,4...|               0.0|
|         5|[(1000,[0,13,21,2...|               0.0|
|         9|[(1000,[5,6,46,47...|               0.0|
+----------+--------------------+------------------+
only showing top 5 rows



In [50]:
#Exercise 3.3 d)
#Creating a list of inter-cluster distances.
interClust10List = interClustDist10DF.select("prediction", "inter_cluster_dist")\
                                    .orderBy("prediction", ascending=True)\
                                    .rdd.map(lambda x: x[1]).collect()
#interClust10List

[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 17.07427978515625, 26.09011459350586, 0.0]

In [51]:
#Exercise 3.3 d)
#Davies-Bouldin Index for a cluster of size 10
calcDBIndex(centroids10_list, interClust10List)

0.9387042199224245

## Exercise 3.4 (Latent Direchlet Allocation (LDA))

a) Running LDA Algorithm with k=40 and showing the top 5 terms for each extracted topic.

In [151]:
#Exercise 3.3 a)
#Initializing an LDA object.
lda = LDA(featuresCol="tf_vector", k=40, maxIter=10)
#Creating the model.
ldaModel = lda.fit(featurizedDataDF.select("tf_vector"))

In [152]:
#Exercise 3.3 a)
#Showing the top 5 terms for each extracted latent topic.
topTopics_5 = ldaModel.describeTopics(5)
topTopics_5.show(5, truncate=False)

+-----+-------------------------+------------------------------------------------------------------------------------------------------------------+
|topic|termIndices              |termWeights                                                                                                       |
+-----+-------------------------+------------------------------------------------------------------------------------------------------------------+
|0    |[726, 779, 743, 175, 588]|[0.0013339958833439576, 0.001320473180729142, 0.0012953853029319666, 0.001294426871207656, 0.001291593299519583]  |
|1    |[851, 204, 320, 838, 927]|[0.001317823910699884, 0.001312613168814432, 0.0013094725739818392, 0.0012925885391767084, 0.0012878698150576423] |
|2    |[388, 901, 596, 668, 845]|[0.001314594528894793, 0.0013080723360205605, 0.0013052458040161991, 0.0012926310130867316, 0.0012798505196279092]|
|3    |[38, 738, 220, 420, 830] |[0.0013183847054659037, 0.001302549516896273, 0.0012938533467724634, 0.00

In [158]:
#Exercise 3.3 a)
#Showing the top 5 terms for each extracted latent topic.
topTopics_5.select("topic", Func.explode("termIndices").alias("word_id"))\
.join(termsDF, "word_id" ).show()

+-------+-----+-----------+
|word_id|topic|single_word|
+-------+-----+-----------+
|      0|   12|    process|
|      5|   37|       read|
|      5|   29|       read|
|      5|   11|       read|
|      8|   29|      basic|
|     11|    4|   interest|
|     14|   22|   knowledg|
|     18|   29|       call|
|     23|   20|      power|
|     24|   37|    program|
|     26|   20|       link|
|     29|   28|     explor|
|     30|   19|      singl|
|     34|   18|     scienc|
|     35|   33|      learn|
|     36|   37|      align|
|     38|    3|     number|
|     40|   29|    collect|
|     45|   10|     featur|
|     46|   30|     provid|
+-------+-----+-----------+
only showing top 20 rows



In [159]:
#Exercise 3.3 a)
#Creating a dataframe with the topic distibution generated from LDA.
ldaTransformed = ldaModel.transform(featurizedDataDF)
ldaTransformed.first()

Row(paper_id=115158, tf_vector=SparseVector(1000, {0: 1.0, 1: 1.0, 12: 1.0, 13: 1.0, 21: 1.0, 22: 2.0, 24: 1.0, 27: 1.0, 31: 1.0, 37: 1.0, 43: 1.0, 51: 1.0, 55: 1.0, 62: 1.0, 69: 1.0, 70: 1.0, 76: 1.0, 79: 1.0, 80: 1.0, 84: 1.0, 90: 1.0, 92: 1.0, 95: 1.0, 105: 2.0, 109: 1.0, 110: 1.0, 111: 2.0, 113: 1.0, 117: 1.0, 120: 1.0, 127: 1.0, 130: 1.0, 145: 1.0, 162: 1.0, 164: 1.0, 169: 1.0, 174: 1.0, 180: 1.0, 202: 1.0, 209: 1.0, 219: 1.0, 222: 1.0, 224: 1.0, 239: 1.0, 245: 1.0, 246: 1.0, 258: 1.0, 263: 2.0, 270: 1.0, 275: 1.0, 317: 2.0, 329: 1.0, 336: 1.0, 339: 1.0, 355: 1.0, 364: 1.0, 369: 1.0, 377: 1.0, 384: 1.0, 387: 1.0, 405: 1.0, 411: 1.0, 418: 2.0, 423: 1.0, 436: 1.0, 440: 1.0, 450: 1.0, 454: 1.0, 459: 1.0, 471: 1.0, 500: 1.0, 534: 1.0, 570: 1.0, 583: 1.0, 584: 1.0, 596: 1.0, 614: 1.0}), topicDistribution=DenseVector([0.0003, 0.0003, 0.0003, 0.0003, 0.0003, 0.0003, 0.0003, 0.0003, 0.0003, 0.0003, 0.0003, 0.0003, 0.0003, 0.0003, 0.0003, 0.0003, 0.0003, 0.0003, 0.0003, 0.0003, 0.0003, 0.0

In [160]:
#Exercise 3.3 a)
#Joining the topic distribution to the users.
joinUserLDA = explodUsersDF.join(ldaTransformed, "paper_id")
joinUserLDA.show(5, truncate=True)

+--------+--------------------+--------------------+--------------------+
|paper_id|        user_hash_id|           tf_vector|   topicDistribution|
+--------+--------------------+--------------------+--------------------+
|  740681|b656009a6efdc8b1a...|(1000,[0,14,22,37...|[8.70247031601365...|
| 4778506|a06e5e3519a8d90a8...|(1000,[0,1,5,33,3...|[8.70247031601365...|
|  105906|d2e2a082f925839b3...|(1000,[0,69,292,5...|[0.00488615736766...|
|  105906|f1e1cd4ff25018273...|(1000,[0,69,292,5...|[0.00488615736766...|
|  212874|a06e5e3519a8d90a8...|(1000,[0,1,14,37,...|[8.70247031601365...|
+--------+--------------------+--------------------+--------------------+
only showing top 5 rows



In [161]:
#Exercise 3.3 a)
#Grouping the topic distribution with respect to the users.
grpUserTopicDistDF = joinUserLDA.groupBy("user_hash_id")\
                                .agg(Func.collect_list("topicDistribution").alias("topic_distribution_ls"))
grpUserTopicDistDF.first()

Row(user_hash_id='f1e1cd4ff25018273aafc0c68fbb5a2f', topic_distribution_ls=[DenseVector([0.0049, 0.0049, 0.0049, 0.0049, 0.0049, 0.0049, 0.0049, 0.0049, 0.0049, 0.0049, 0.0049, 0.0049, 0.0049, 0.0049, 0.0049, 0.0049, 0.0049, 0.0049, 0.0049, 0.0049, 0.0053, 0.0049, 0.8077, 0.0049, 0.0049, 0.0049, 0.0049, 0.0049, 0.0049, 0.006, 0.0049, 0.0049, 0.0049, 0.005, 0.0049, 0.0049, 0.0049, 0.005, 0.0049, 0.0049]), DenseVector([0.0061, 0.0061, 0.0061, 0.0061, 0.0061, 0.0061, 0.0061, 0.0061, 0.0061, 0.0061, 0.0061, 0.0061, 0.0061, 0.0061, 0.0061, 0.0061, 0.0061, 0.0061, 0.0061, 0.0061, 0.0066, 0.0061, 0.0064, 0.0061, 0.0061, 0.0061, 0.0061, 0.0061, 0.0061, 0.7605, 0.0061, 0.0061, 0.0061, 0.0062, 0.0061, 0.0061, 0.0061, 0.0063, 0.0061, 0.0061]), DenseVector([0.0009, 0.0009, 0.0009, 0.0009, 0.0009, 0.0009, 0.0009, 0.0009, 0.0009, 0.0009, 0.0009, 0.0009, 0.0009, 0.0009, 0.0009, 0.0009, 0.0009, 0.0009, 0.0009, 0.0009, 0.0009, 0.0009, 0.0009, 0.0009, 0.0009, 0.0009, 0.0009, 0.0009, 0.0009, 0.9659, 0.00

b) Calculating the LDA based user profiles as the summation of the paper topics vectors.

In [162]:
#Exercise 3.4 b)
#Function to sum a list of dense vectors.
def addDenseVec(dense_vec_list):
    result = DenseVector(np.zeros(40))
    for dense_vector in dense_vec_list:
        result = np.add(result, dense_vector).tolist() 
    return DenseVector(result)

#Creating a user defined function.
udf_addDenseVec = Func.udf(addDenseVec, VectorUDT())

In [163]:
#Exercise 3.4 b)
#Summation of the paper topics vectors.
userLDAProfile = grpUserTopicDistDF.withColumn("features",\
                                               Func.lit(udf_addDenseVec(grpUserTopicDistDF.topic_distribution_ls)))

userLDAProfile.select("user_hash_id", "features").first()

Row(user_hash_id='f1e1cd4ff25018273aafc0c68fbb5a2f', features=DenseVector([0.0168, 0.0168, 0.0168, 0.0168, 0.0168, 0.0168, 0.0168, 0.0168, 0.0168, 0.0168, 0.0168, 0.0168, 0.0168, 0.0168, 0.0168, 0.0168, 0.0168, 0.0168, 0.0168, 0.0168, 0.0181, 0.0168, 0.8201, 0.0168, 0.0168, 0.0168, 0.0168, 0.0168, 0.0169, 2.5408, 0.0168, 0.0168, 0.0168, 0.017, 0.0168, 0.0168, 0.0168, 0.0173, 0.0168, 0.0168]))

c) Applying the K-means algorithm to cluster the users using their LDA profiles in 50 clusters.

In [164]:
#Exercise 3.4 c)
#Initializing the K-means object.
UsrLDAKmeans = KMeans(featuresCol="features", k=50, maxIter=10)
#Creating the model.
userLDAClustModel = UsrLDAKmeans.fit(userLDAProfile.select("features"))

In [165]:
#Exercise 3.4 d)
#Getting a list of centroids.
topicCentroids_list = userLDAClustModel.clusterCenters()
topicCentroids_list[0:1]

[array([0.00243919, 0.00243919, 0.00243919, 0.00243919, 0.00243919,
        0.00243919, 0.00243919, 0.00243919, 0.00243919, 0.00245016,
        0.00243919, 0.00243919, 0.00243919, 0.00243919, 0.00243919,
        0.00243919, 0.00243919, 0.00243919, 0.00243919, 0.00245023,
        0.00263719, 0.00243919, 0.0025533 , 0.00243919, 0.00243919,
        0.00243919, 0.00243919, 0.00243919, 0.00245382, 0.90441067,
        0.00243919, 0.00243919, 0.00243919, 0.00247763, 0.00243919,
        0.00243919, 0.00243919, 0.00251302, 0.00243919, 0.00243919])]

In [166]:
#Exercise 3.4 d)
#Creating a dataframe with the predictions of the KMeans algo.
userTopicTransformed = userLDAClustModel.transform(userLDAProfile)
userTopicTransformed.show(5)

+--------------------+---------------------+--------------------+----------+
|        user_hash_id|topic_distribution_ls|            features|prediction|
+--------------------+---------------------+--------------------+----------+
|f1e1cd4ff25018273...| [[0.0048861573676...|[0.01675513477498...|         1|
|cf9c7f356092c34be...| [[7.6141206089542...|[7.61412060895422...|         2|
|f3da73f972caebbe0...| [[0.0024391869663...|[0.00243918696632...|         0|
|d46f0fdd50ec8aa79...| [[7.8598634786179...|[7.85986347861791...|         3|
|d503571e44a0373eb...| [[7.8598634786179...|[7.85986347861791...|         3|
+--------------------+---------------------+--------------------+----------+
only showing top 5 rows



In [167]:
#Exercise 3.4 d)
#Grouping the user features with respect to the prediction.
grpUserTopicProfDF = userTopicTransformed.groupBy("prediction")\
                                            .agg(Func.collect_list("features").alias("paper_topic_pts"))
grpUserTopicProfDF.show(5)

+----------+--------------------+
|prediction|     paper_topic_pts|
+----------+--------------------+
|        12|[[0.0011627482152...|
|        22|[[8.7024703160136...|
|         1|[[0.0167551347749...|
|        13|[[0.0013224931929...|
|         6|[[0.0130471140319...|
+----------+--------------------+
only showing top 5 rows



In [168]:
#Exercise 3.4 d)
#Creating a user defined function.
udf_calcTopicInterClust = Func.udf(lambda x, y: interClusterDist(x, y, topicCentroids_list), FloatType())

In [169]:
#Exercise 3.4 d)
#Creating a dataframe containing the inter-cluster distance values.
topicInterClustDist = grpUserTopicProfDF.withColumn("inter_cluster_dist",\
                                                udf_calcTopicInterClust(grpUserTopicProfDF.prediction,\
                                                                        grpUserTopicProfDF.paper_topic_pts))
topicInterClustDist.show(10)

+----------+--------------------+------------------+
|prediction|     paper_topic_pts|inter_cluster_dist|
+----------+--------------------+------------------+
|        12|[[0.0011627482152...|               0.0|
|        22|[[8.7024703160136...|               0.0|
|         1|[[0.0167551347749...|               0.0|
|        13|[[0.0013224931929...|               0.0|
|         6|[[0.0130471140319...|               0.0|
|        16|[[0.0109467675524...|               0.0|
|         3|[[7.8598634786179...|     2.4243498E-19|
|        20|[[0.0363485545860...|               0.0|
|         5|[[0.0022598097566...|               0.0|
|        19|[[0.0048861573676...|               0.0|
+----------+--------------------+------------------+
only showing top 10 rows



In [170]:
#Exercise 3.4 d)
#Creating a list of inter-cluster distances.
topicInterClustList = topicInterClustDist.select("prediction", "inter_cluster_dist")\
                                    .orderBy("prediction", ascending=True)\
                                    .rdd.map(lambda x: x[1]).collect()

In [171]:
#Exercise 3.4 d)
#Davies-Bouldin Index for a cluster of size 50
calcDBIndex(topicCentroids_list, topicInterClustList)

0.005571902894568843